In [3]:
!pip install langchain streamlit faiss-cpu pytesseract pillow openai whisper


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 179.3 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 774.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.6 MB/s eta 0:00:00
  Created wheel for whisper: filename=whisper-1.1.10-py3-none-any.whl size=41120 sha256=7dc489b3874728082f534977745da85370b4d636430241970926dbd5cfed1eec
  Stored in directory: /root/.cache/pip/wheels/aa/7c/1d/015619716e2facae6631312503baf3c3220e6a9a3508cb14b6
Successfully built whisper


In [4]:
from google.colab import files
import zipfile
import os

# Upload the zip file
uploaded = files.upload()

# Extract the uploaded zip file
for filename in uploaded.keys():
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall("/content/uploaded_data")
print("Files extracted to /content/uploaded_data")
print("Contents:", os.listdir("/content/uploaded_data"))


Saving sample_data.zip to sample_data.zip
Files extracted to /content/uploaded_data
Contents: ['FAQs.txt', 'UserGuide.pdf', 'ErrorLogs.log', 'ProcessFlow.png']


In [5]:
%%writefile app.py
import os
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from PIL import Image
import pytesseract
import streamlit as st

# Data preprocessing functions
def preprocess_text_files(directory):
    splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    documents = []
    for file in os.listdir(directory):
        if file.endswith('.txt'):
            loader = TextLoader(os.path.join(directory, file))
        elif file.endswith('.pdf'):
            loader = PyPDFLoader(os.path.join(directory, file))
        else:
            continue
        docs = loader.load_and_split(splitter)
        documents.extend(docs)
    return documents

def process_images(directory):
    text_data = []
    for file in os.listdir(directory):
        if file.endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(directory, file)
            text = pytesseract.image_to_string(Image.open(image_path))
            text_data.append(text)
    return text_data

def process_audio_video(directory):
    import whisper
    model = whisper.load_model("base")
    transcripts = []
    for file in os.listdir(directory):
        if file.endswith(('.mp4', '.mp3')):
            result = model.transcribe(os.path.join(directory, file))
            transcripts.append(result["text"])
    return transcripts

# Create vector store
def create_vector_store(documents):
    embeddings = OpenAIEmbeddings()
    vector_store = FAISS.from_documents(documents, embeddings)
    return vector_store

# RAG workflow
def rag_workflow(query, vector_store):
    retriever = vector_store.as_retriever()
    qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(), retriever=retriever)
    response = qa_chain.run(query)
    return response

# Streamlit app
st.title("RAG System")
st.header("Ask Questions from the Knowledge Base")

uploaded_files_dir = "/content/uploaded_data"  # Colab-specific directory

if "vector_store" not in st.session_state:
    st.write("Preprocessing data and building vector store...")
    text_docs = preprocess_text_files(uploaded_files_dir)
    image_texts = process_images(uploaded_files_dir)
    audio_transcripts = process_audio_video(uploaded_files_dir)

    # Combine all data
    documents = text_docs + [{"text": text} for text in image_texts + audio_transcripts]
    st.session_state["vector_store"] = create_vector_store(documents)
    st.success("Vector store created!")

query = st.text_input("Enter your query:")
if query:
    response = rag_workflow(query, st.session_state["vector_store"])
    st.write("Response:", response)


Writing app.py


In [6]:
!pip install streamlit


In [7]:
!nohup streamlit run app.py &


nohup: appending output to 'nohup.out'


In [11]:
!python3 -m http.server 8503 &


Serving HTTP on 0.0.0.0 port 8503 (http://0.0.0.0:8503/) ...


KeyboardInterrupt: 

In [13]:
!npm install -g localtunnel


⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙
added 22 packages in 2s
⠙
⠙3 packages are looking for funding
⠙  run `npm fund` for details
⠙

In [14]:
!curl ifconfig.me


34.19.91.72

In [15]:
!lt --port 8501


your url is: https://rich-socks-check.loca.lt
^C
